# Single Particle performance plots


   * [#$p_{T}$ response](#$p_{T}$-response)


In [15]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

In [16]:
# %load ../drawingTools
import ROOT
import math
import uuid

# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 12)
stuff = []
f_idx = 0

ROOT.gStyle.SetOptTitle(False)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.13)

# ROOT.gStyle.SetCanvasBorderMode(0)
# ROOT.gStyle.SetCanvasColor(0)


def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext

def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.05)
    return legend

def newCanvas(name=None, title=None, height=600, width=800, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title, width, height
    canvas = ROOT.TCanvas(name, title, width, height)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas

def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')

    c.Draw()


def draw(histograms,
         labels,
         options='',
         norm=False,
         logy=False,
         min_y=None,
         max_y=None,
         text=None,
         y_axis_label=None,
         x_axis_label=None,
         v_lines=None,
         h_lines=None,
         do_stats=False,
         do_profile=False,
         do_ratio=False):
    global colors
    global stuff
    global p_idx


    # 0 - determine kind and # of histos
    n_histos = len(histograms)
    if n_histos == 0:
        print '[draw]**ERROR: empy histo list'
        return

    histo_class = histograms[0].ClassName()

    # 1 - check argument consisntency
    if 'TH2' not in histo_class and do_profile:
        print '[draw]**ERROR: do_profile set for histo of class: {}'.format(histo_class)
        return

    # TH1 histograms are overlayed
    # TH2 histograms are drawn side by side in the same canvas
    overlay = True
    if n_histos > 1:
        if 'TH2' in histo_class or 'TH3' in histo_class:
            overlay = False

    # 2 - determine canvas properties
    xdiv = 0
    ydiv = 0
    c_width = 800
    c_height = 600
    # do_ratio = False

    if do_ratio:
        c_height = 800

    if not overlay:
        xdiv = 2
        c_width = 1000.
        ydiv = math.ceil(float(n_histos)/2)
        c_height = 500*ydiv
    canvas = newCanvas(name=None,
                       title=None,
                       height=int(c_height),
                       width=int(c_width),
                       xdiv=int(xdiv),
                       ydiv=int(ydiv))
    if overlay:
        canvas.SetRightMargin(0.30)

    canvas.cd()
    leg = getLegend()

    drawn_histos = []
    # drawn_histos = histograms
    for hidx, hist in enumerate(histograms):
        opt = options
        if 'TGraph' not in histo_class:
            hist.SetStats(do_stats)
        else:
            opt = 'P'+options
        if overlay:
            hist.SetLineColor(colors[hidx])
            if 'TGraph' in histo_class:
                hist.SetMarkerColor(colors[hidx])
            if hidx:
                opt = 'same,'+opt
            else:
                if 'TGraph' in histo_class:
                    opt = opt+'A'
        else:
            canvas.cd(hidx+1)
        # print hidx, opt
        d_hist = hist
        if norm:
            d_hist = hist.DrawNormalized(opt, 1.)
        else:
            hist.Draw(opt)

        if do_profile:
            profname = d_hist.GetName()+'_prof_'+str(p_idx)
            p_idx += 1
            firstbin = 1
            lastbin = -1
            prof = d_hist.ProfileX(profname, firstbin, lastbin, 's')
            prof.SetMarkerColor(2)
            prof.SetLineColor(2)
            prof.Draw('same')

        if overlay:
            if 'TGraph' not in histo_class:
                leg.AddEntry(hist, labels[hidx], 'l')
            else:
                leg.AddEntry(hist, labels[hidx], 'P')
        else:
            if text:
                newtext = '{}: {}'.format(labels[hidx], text)
                rtext = getText(newtext, 0.15, 0.85)
                rtext.Draw('same')
        drawn_histos.append(d_hist)

    if do_ratio:
        h_ratio = ROOT.TRatioPlot(drawn_histos[0], drawn_histos[1])
        stuff.append(h_ratio)
        h_ratio.Draw()

        pad = canvas.cd(1)
        h_ratio.GetUpperPad().SetBottomMargin(0)
        h_ratio.GetUpperPad().SetRightMargin(0.3)
        if logy:
            h_ratio.GetUpperPad().SetLogy()
        pad.Update()
        pad = canvas.cd(2)
        h_ratio.GetLowerPad().SetTopMargin(0)
        h_ratio.GetLowerPad().SetRightMargin(0.3)
        pad.Update()
        # hratio.GetLowerRefGraph().GetYaxis().SetRangeUser(-6, 6)

    canvas.Update()

    # we now set the axis properties
    max_value = max_y
    min_value = min_y

    if min_y is None:
        min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in drawn_histos])
    if max_y is None:
        max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in drawn_histos])*1.2
    # print min_value, max_value

    for hist in drawn_histos:
        hist.GetYaxis().SetRangeUser(min_value, max_value)
        if y_axis_label:
            hist.GetYaxis().SetTitle(y_axis_label)
        if x_axis_label:
            hist.GetXaxis().SetTitle(x_axis_label)

    canvas.Draw()

    # we draw additional stuff if needed
    if overlay:
        canvas.cd()
        leg.Draw()
        if text:
            rtext = getText(text, 0.15, 0.85)
            rtext.Draw("same")

    pad_range = range(0, 1)
    if not overlay:
        pad_range = range(1, len(histograms)+1)

    for pad_id in pad_range:
        canvas.cd(pad_id)
        if v_lines:
            for v_line_x in v_lines:
                aline = ROOT.TLine(v_line_x, ROOT.gPad.GetUymin(), v_line_x,  ROOT.gPad.GetUymax())
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if h_lines:
            for h_line_y in h_lines:
                aline = ROOT.TLine(ROOT.gPad.GetUxmin(), h_line_y, ROOT.gPad.GetUxmax(),  h_line_y)
                aline.SetLineStyle(2)
                aline.Draw("same")
                stuff.append(aline)
        if logy:
            if not do_ratio:
                ROOT.gPad.SetLogy()

        ROOT.gPad.Update()
    canvas.Update()
    canvas.Draw()

    return canvas


In [18]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v66'

files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, type=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = ROOT.TFile(self.histo_filename, 'r')
        self.type = type


# sample_names = ['ele_flat2to100_PU0',
#                 'ele_flat2to100_PU200',
#                 'photonPt35_PU0',
#                 'photonPt35_PU200']


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            self.instance = self.classtype(name, self.root_file)
        return self.instance


class HPlot:
    def __init__(self, samples, tp_sets, tp_selections, gen_selections):
        self.tp_sets = tp_sets
        self.tp_selections = tp_selections
        self.gen_selections = gen_selections
        self.pus = []
        self.labels_dict = {}

        for sample in samples:
            self.pus.append(sample.label)
            self.labels_dict[sample.type] = sample.type

        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])

        self.labels_dict.update(tp_sets)
        self.labels_dict.update(tp_selections)
        self.labels_dict.update(gen_selections)
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

    def cache_histo(self,
                    classtype,
                    samples,
                    pus,
                    tps,
                    tp_sels,
                    gen_sels):
        if gen_sels is None:
            gen_sels = [None]

        for sample in samples:
            print sample
            for tp in tps:
                for tp_sel in tp_sels:
                    for gen_sel in gen_sels:
                        print sample, tp, tp_sel, gen_sel
                        self.data = self.data.append({'sample': sample.type,
                                                      'pu': sample.label,
                                                      'tp': tp,
                                                      'tp_sel': tp_sel,
                                                      'gen_sel': gen_sel,
                                                      'classtype': classtype,
                                                      'histo': HProxy(classtype, tp, tp_sel, gen_sel, sample.histo_file)},
                                                     ignore_index=True)

    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None):
        histo = None
        labels = []
        text = ''

        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += ' & (gen_sel == @gen_sel)'
        else:
            query += ' & (gen_sel.isnull())'
        if sample is not None:
            query += '& (sample == @sample)'

        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
#         print histo_df

        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get() for his in histo_df['histo'].values]
        return histo, labels, text


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version+'t', 'ele')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

tpsets = {'DEF': 'dRC3d',
          'DEFMerged': 'dRC3d (merged)',
          'HMvDR': 'HMC3d+dR(layer)',
          'HMvDRMerged': 'HMC3d+dR(layer)+merged',}

tpset_selections = {}
gen_selections = {}
samples = []

# tpset_selections.update(get_label_dict(tp_rate_selections))
tpset_selections.update(get_label_dict(selections.tp_match_selections))
gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})


Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_v66.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_v66.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v66.root does not exist


In [19]:
tpsets


{'DEF': 'dRC3d',
 'DEFMerged': 'dRC3d (merged)',
 'HMvDR': 'HMC3d+dR(layer)',
 'HMvDRMerged': 'HMC3d+dR(layer)+merged'}

In [20]:
# tpset_selections.pop('Emv1')
tpset_selections


{'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'Pt10': 'p_{T}^{L1}>=10GeV',
 'Pt20': 'p_{T}^{L1}>=20GeV',
 'Pt30': 'p_{T}^{L1}>=30GeV',
 'all': ''}

In [21]:
gen_selections

{'GEN': '',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'nomatch': ''}

In [7]:
files

{}

In [22]:
%%time
samples = samples_ele
samples+= samples_photons


hplot = HPlot(samples, tpsets, tpset_selections, gen_selections)


hplot.cache_histo(classtype=histos.ClusterConeHistos, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=gen_selections)


# print samples
# for sample in samples:
#     print sample
#     for tp in tpsets:
#         for tp_sel in tpset_selections:
#             for gen_sel in gen_selections:
#                 print sample, tp, tp_sel, gen_sel
#                 hplot.data = hplot.data.append({'sample': 'ele',
#                                                 'pu': sample.label,
#                                                 'tp': tp,
#                                                 'tp_sel': tp_sel,
#                                                 'gen_sel': gen_sel,
#                                                 'classtype': histos.HistoSetReso,
#                                                 'histo': histos.HistoSetReso('{}_{}_{}'.format(tp, tp_sel, gen_sel), sample.histo_file)}
#                                               , ignore_index=True)


<__main__.Sample instance at 0x11f6e77e8>
<__main__.Sample instance at 0x11f6e77e8> DEFMerged Em GENPt40
<__main__.Sample instance at 0x11f6e77e8> DEFMerged Em GENEtaC
<__main__.Sample instance at 0x11f6e77e8> DEFMerged Em GENEtaB
<__main__.Sample instance at 0x11f6e77e8> DEFMerged Em GENEtaD
<__main__.Sample instance at 0x11f6e77e8> DEFMerged Em GENEtaBC
<__main__.Sample instance at 0x11f6e77e8> DEFMerged Em GENPt30
<__main__.Sample instance at 0x11f6e77e8> DEFMerged Em GENPt20
<__main__.Sample instance at 0x11f6e77e8> DEFMerged Em GENEtaBCD
<__main__.Sample instance at 0x11f6e77e8> DEFMerged Em GENPt10
<__main__.Sample instance at 0x11f6e77e8> DEFMerged Em nomatch
<__main__.Sample instance at 0x11f6e77e8> DEFMerged Em GEN
<__main__.Sample instance at 0x11f6e77e8> DEFMerged all GENPt40
<__main__.Sample instance at 0x11f6e77e8> DEFMerged all GENEtaC
<__main__.Sample instance at 0x11f6e77e8> DEFMerged all GENEtaB
<__main__.Sample instance at 0x11f6e77e8> DEFMerged all GENEtaD
<__main__.

<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt30 GENEtaB
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt30 GENEtaD
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt30 GENEtaBC
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt30 GENPt30
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt30 GENPt20
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt30 GENEtaBCD
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt30 GENPt10
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt30 nomatch
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt30 GEN
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt20 GENPt40
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt20 GENEtaC
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt20 GENEtaB
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt20 GENEtaD
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt20 GENEtaBC
<__main__.Sample instance at 0x11f6e77e8> HMvDRMerged Pt20 GEN

<__main__.Sample instance at 0x11f6e77e8> DEF all GENEtaD
<__main__.Sample instance at 0x11f6e77e8> DEF all GENEtaBC
<__main__.Sample instance at 0x11f6e77e8> DEF all GENPt30
<__main__.Sample instance at 0x11f6e77e8> DEF all GENPt20
<__main__.Sample instance at 0x11f6e77e8> DEF all GENEtaBCD
<__main__.Sample instance at 0x11f6e77e8> DEF all GENPt10
<__main__.Sample instance at 0x11f6e77e8> DEF all nomatch
<__main__.Sample instance at 0x11f6e77e8> DEF all GEN
<__main__.Sample instance at 0x11f6e77e8> DEF Pt10 GENPt40
<__main__.Sample instance at 0x11f6e77e8> DEF Pt10 GENEtaC
<__main__.Sample instance at 0x11f6e77e8> DEF Pt10 GENEtaB
<__main__.Sample instance at 0x11f6e77e8> DEF Pt10 GENEtaD
<__main__.Sample instance at 0x11f6e77e8> DEF Pt10 GENEtaBC
<__main__.Sample instance at 0x11f6e77e8> DEF Pt10 GENPt30
<__main__.Sample instance at 0x11f6e77e8> DEF Pt10 GENPt20
<__main__.Sample instance at 0x11f6e77e8> DEF Pt10 GENEtaBCD
<__main__.Sample instance at 0x11f6e77e8> DEF Pt10 GENPt10
<__

<__main__.Sample instance at 0x11f6e7710> DEFMerged EmPt10 GENEtaBCD
<__main__.Sample instance at 0x11f6e7710> DEFMerged EmPt10 GENPt10
<__main__.Sample instance at 0x11f6e7710> DEFMerged EmPt10 nomatch
<__main__.Sample instance at 0x11f6e7710> DEFMerged EmPt10 GEN
<__main__.Sample instance at 0x11f6e7710> HMvDRMerged Em GENPt40
<__main__.Sample instance at 0x11f6e7710> HMvDRMerged Em GENEtaC
<__main__.Sample instance at 0x11f6e7710> HMvDRMerged Em GENEtaB
<__main__.Sample instance at 0x11f6e7710> HMvDRMerged Em GENEtaD
<__main__.Sample instance at 0x11f6e7710> HMvDRMerged Em GENEtaBC
<__main__.Sample instance at 0x11f6e7710> HMvDRMerged Em GENPt30
<__main__.Sample instance at 0x11f6e7710> HMvDRMerged Em GENPt20
<__main__.Sample instance at 0x11f6e7710> HMvDRMerged Em GENEtaBCD
<__main__.Sample instance at 0x11f6e7710> HMvDRMerged Em GENPt10
<__main__.Sample instance at 0x11f6e7710> HMvDRMerged Em nomatch
<__main__.Sample instance at 0x11f6e7710> HMvDRMerged Em GEN
<__main__.Sample ins

<__main__.Sample instance at 0x11f6e7710> HMvDR Pt30 nomatch
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt30 GEN
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt20 GENPt40
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt20 GENEtaC
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt20 GENEtaB
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt20 GENEtaD
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt20 GENEtaBC
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt20 GENPt30
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt20 GENPt20
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt20 GENEtaBCD
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt20 GENPt10
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt20 nomatch
<__main__.Sample instance at 0x11f6e7710> HMvDR Pt20 GEN
<__main__.Sample instance at 0x11f6e7710> HMvDR EmPt10 GENPt40
<__main__.Sample instance at 0x11f6e7710> HMvDR EmPt10 GENEtaC
<__main__.Sample instance at 0x11f6e7710> HMvDR EmPt10 GENEtaB
<__main__.Sample instan

<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt30 GENPt20
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt30 GENEtaBCD
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt30 GENPt10
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt30 nomatch
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt30 GEN
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt20 GENPt40
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt20 GENEtaC
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt20 GENEtaB
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt20 GENEtaD
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt20 GENEtaBC
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt20 GENPt30
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt20 GENPt20
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt20 GENEtaBCD
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt20 GENPt10
<__main__.Sample instance at 0x11f6e77a0> DEFMerged EmPt20 no

<__main__.Sample instance at 0x11f6e77a0> HMvDR Em GENPt40
<__main__.Sample instance at 0x11f6e77a0> HMvDR Em GENEtaC
<__main__.Sample instance at 0x11f6e77a0> HMvDR Em GENEtaB
<__main__.Sample instance at 0x11f6e77a0> HMvDR Em GENEtaD
<__main__.Sample instance at 0x11f6e77a0> HMvDR Em GENEtaBC
<__main__.Sample instance at 0x11f6e77a0> HMvDR Em GENPt30
<__main__.Sample instance at 0x11f6e77a0> HMvDR Em GENPt20
<__main__.Sample instance at 0x11f6e77a0> HMvDR Em GENEtaBCD
<__main__.Sample instance at 0x11f6e77a0> HMvDR Em GENPt10
<__main__.Sample instance at 0x11f6e77a0> HMvDR Em nomatch
<__main__.Sample instance at 0x11f6e77a0> HMvDR Em GEN
<__main__.Sample instance at 0x11f6e77a0> HMvDR all GENPt40
<__main__.Sample instance at 0x11f6e77a0> HMvDR all GENEtaC
<__main__.Sample instance at 0x11f6e77a0> HMvDR all GENEtaB
<__main__.Sample instance at 0x11f6e77a0> HMvDR all GENEtaD
<__main__.Sample instance at 0x11f6e77a0> HMvDR all GENEtaBC
<__main__.Sample instance at 0x11f6e77a0> HMvDR all

<__main__.Sample instance at 0x11f6e77a0> DEF EmPt20 GEN
<__main__.Sample instance at 0x11f6e77a0> DEF Pt30 GENPt40
<__main__.Sample instance at 0x11f6e77a0> DEF Pt30 GENEtaC
<__main__.Sample instance at 0x11f6e77a0> DEF Pt30 GENEtaB
<__main__.Sample instance at 0x11f6e77a0> DEF Pt30 GENEtaD
<__main__.Sample instance at 0x11f6e77a0> DEF Pt30 GENEtaBC
<__main__.Sample instance at 0x11f6e77a0> DEF Pt30 GENPt30
<__main__.Sample instance at 0x11f6e77a0> DEF Pt30 GENPt20
<__main__.Sample instance at 0x11f6e77a0> DEF Pt30 GENEtaBCD
<__main__.Sample instance at 0x11f6e77a0> DEF Pt30 GENPt10
<__main__.Sample instance at 0x11f6e77a0> DEF Pt30 nomatch
<__main__.Sample instance at 0x11f6e77a0> DEF Pt30 GEN
<__main__.Sample instance at 0x11f6e77a0> DEF Pt20 GENPt40
<__main__.Sample instance at 0x11f6e77a0> DEF Pt20 GENEtaC
<__main__.Sample instance at 0x11f6e77a0> DEF Pt20 GENEtaB
<__main__.Sample instance at 0x11f6e77a0> DEF Pt20 GENEtaD
<__main__.Sample instance at 0x11f6e77a0> DEF Pt20 GENEtaBC

<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged Pt10 GENEtaB
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged Pt10 GENEtaD
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged Pt10 GENEtaBC
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged Pt10 GENPt30
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged Pt10 GENPt20
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged Pt10 GENEtaBCD
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged Pt10 GENPt10
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged Pt10 nomatch
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged Pt10 GEN
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged EmPt30 GENPt40
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged EmPt30 GENEtaC
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged EmPt30 GENEtaB
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged EmPt30 GENEtaD
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerged EmPt30 GENEtaBC
<__main__.Sample instance at 0x11f6e78c0> HMvDRMerge

<__main__.Sample instance at 0x11f6e78c0> DEF Em GENPt40
<__main__.Sample instance at 0x11f6e78c0> DEF Em GENEtaC
<__main__.Sample instance at 0x11f6e78c0> DEF Em GENEtaB
<__main__.Sample instance at 0x11f6e78c0> DEF Em GENEtaD
<__main__.Sample instance at 0x11f6e78c0> DEF Em GENEtaBC
<__main__.Sample instance at 0x11f6e78c0> DEF Em GENPt30
<__main__.Sample instance at 0x11f6e78c0> DEF Em GENPt20
<__main__.Sample instance at 0x11f6e78c0> DEF Em GENEtaBCD
<__main__.Sample instance at 0x11f6e78c0> DEF Em GENPt10
<__main__.Sample instance at 0x11f6e78c0> DEF Em nomatch
<__main__.Sample instance at 0x11f6e78c0> DEF Em GEN
<__main__.Sample instance at 0x11f6e78c0> DEF all GENPt40
<__main__.Sample instance at 0x11f6e78c0> DEF all GENEtaC
<__main__.Sample instance at 0x11f6e78c0> DEF all GENEtaB
<__main__.Sample instance at 0x11f6e78c0> DEF all GENEtaD
<__main__.Sample instance at 0x11f6e78c0> DEF all GENEtaBC
<__main__.Sample instance at 0x11f6e78c0> DEF all GENPt30
<__main__.Sample instance

In [9]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,ele,PU0,DEFMerged,Em,GENPt40,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11c7ad290>
1,ele,PU0,DEFMerged,Em,GENEtaC,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11c7adbd8>
2,ele,PU0,DEFMerged,Em,GENEtaB,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11c7ad560>
3,ele,PU0,DEFMerged,Em,GENEtaD,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11c7add40>
4,ele,PU0,DEFMerged,Em,GENEtaBC,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11c7adc68>
5,ele,PU0,DEFMerged,Em,GENPt30,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11c78f710>
6,ele,PU0,DEFMerged,Em,GENPt20,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11c7ad488>
7,ele,PU0,DEFMerged,Em,GENEtaBCD,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11c78f0e0>
8,ele,PU0,DEFMerged,Em,GENPt10,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11c78fdd0>
9,ele,PU0,DEFMerged,Em,nomatch,python.l1THistos.ClusterConeHistos,<__main__.HProxy instance at 0x11c7dc5a8>


In [10]:
hplot.labels_dict

{'DEF': 'dRC3d',
 'DEFMerged': 'dRC3d (merged)',
 'Em': 'EGId',
 'EmPt10': 'EGId, p_{T}^{L1}>=10GeV',
 'EmPt20': 'EGId, p_{T}^{L1}>=20GeV',
 'EmPt30': 'EGId, p_{T}^{L1}>=30GeV',
 'GEN': '',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'HM': 'HistoMaxC3d',
 'HMvDR': 'HistoMaxC3d + dR(layer)',
 'PU0': 'PU0',
 'PU200': 'PU200',
 'Pt10': 'p_{T}^{L1}>=10GeV',
 'Pt20': 'p_{T}^{L1}>=20GeV',
 'Pt30': 'p_{T}^{L1}>=30GeV',
 'all': '',
 'ele': 'ele',
 'nomatch': '',
 'photon': 'photon'}

In [23]:
# tps = ['DEF', 'DEFCalib', 'DEFMerged']
tps = ['DEF', 'HMvDR', 'HMvDRMerged',]

# tp_select = ['all', 'Em' ,'Emv1']
tp_select = ['all', 'Em']

gen_select = ['GENEtaBC', 'GENEtaD']


## Electron samples

In [24]:
sample = 'ele'

In [35]:

for pu in ['PU0', 'PU200']:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, pu, ['DEF', 'HMvDR'], tp_sel, gen_sel)            
            draw([his.h_n for his in hsets], labels, norm=True, text=text, logy=True, do_stats=False, options='hist')
    #             print hsets[0].h_n.GetBinContent(1)


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log s

In [32]:
for gen_sel in gen_select:
    for tp_sel in tp_select:
        for tp in tps:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptRel for his in hsets], labels, norm=True, text=text, options='hist')


In [27]:

for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaEta for his in hsets], labels, norm=True, text=text)


In [28]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaPhi for his in hsets], labels, norm=True, text=text)


In [29]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaPhiVq for his in hsets], labels, text=text, options='COLZ', min_y=-1, max_y=2)


## Photon samples

In [40]:
sample = 'photon'

In [42]:

for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_n for his in hsets], labels, norm=False, text=text, logy=True)


[<ROOT.TH1I object ("DEF_all_GEN_n") at 0x7ff44e3a4290>]
[<ROOT.TH1I object ("DEF_all_GENEtaB_n") at 0x7ff44e4e0a20>]
[<ROOT.TH1I object ("DEF_all_GENEtaBC_n") at 0x7ff44e249750>]
[<ROOT.TH1I object ("DEF_all_GENEtaBCD_n") at 0x7ff44e4eba40>]
[<ROOT.TH1I object ("DEF_all_GENEtaC_n") at 0x7ff44e4f2390>]
[<ROOT.TH1I object ("DEF_all_GENEtaD_n") at 0x7ff44e24f290>]
[<ROOT.TH1I object ("DEF_Em_GEN_n") at 0x7ff44e4f7a90>]
[<ROOT.TH1I object ("DEF_Em_GENEtaB_n") at 0x7ff44e3b0320>]
[<ROOT.TH1I object ("DEF_Em_GENEtaBC_n") at 0x7ff44e3b1d30>]
[<ROOT.TH1I object ("DEF_Em_GENEtaBCD_n") at 0x7ff44e257fd0>]
[<ROOT.TH1I object ("DEF_Em_GENEtaC_n") at 0x7ff44e50c6d0>]
[<ROOT.TH1I object ("DEF_Em_GENEtaD_n") at 0x7ff44e50f510>]
[<ROOT.TH1I object ("DEFMerged_all_GEN_n") at 0x7ff44e518d30>]
[<ROOT.TH1I object ("DEFMerged_all_GENEtaB_n") at 0x7ff44e51b8d0>]
[<ROOT.TH1I object ("DEFMerged_all_GENEtaBC_n") at 0x7ff44e5210f0>]
[<ROOT.TH1I object ("DEFMerged_all_GENEtaBCD_n") at 0x7ff44e3bc5c0>]
[<ROOT.TH

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log s

In [19]:

for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_ptRel for his in hsets], labels, norm=True, text=text)


[<ROOT.TH1F object ("DEF_all_GEN_ptRel") at 0x7ff44e245240>, <ROOT.TH1F object ("DEF_all_GEN_ptRel") at 0x7ff44e245a80>]
[<ROOT.TH1F object ("DEF_all_GENEtaB_ptRel") at 0x7ff44e4e2370>, <ROOT.TH1F object ("DEF_all_GENEtaB_ptRel") at 0x7ff44e4e2f20>]
[<ROOT.TH1F object ("DEF_all_GENEtaBC_ptRel") at 0x7ff44e24a1b0>, <ROOT.TH1F object ("DEF_all_GENEtaBC_ptRel") at 0x7ff44e24ad00>]
[<ROOT.TH1F object ("DEF_all_GENEtaBCD_ptRel") at 0x7ff44dabe000>, <ROOT.TH1F object ("DEF_all_GENEtaBCD_ptRel") at 0x7ff44dabeab0>]
[<ROOT.TH1F object ("DEF_all_GENEtaC_ptRel") at 0x7ff44e3ab1a0>, <ROOT.TH1F object ("DEF_all_GENEtaC_ptRel") at 0x7ff44e3abd50>]
[<ROOT.TH1F object ("DEF_all_GENEtaD_ptRel") at 0x7ff44e4f3c60>, <ROOT.TH1F object ("DEF_all_GENEtaD_ptRel") at 0x7ff44e4f4380>]
[<ROOT.TH1F object ("DEF_Em_GEN_ptRel") at 0x7ff44dac5240>, <ROOT.TH1F object ("DEF_Em_GEN_ptRel") at 0x7ff44dac5df0>]
[<ROOT.TH1F object ("DEF_Em_GENEtaB_ptRel") at 0x7ff44e500a30>, <ROOT.TH1F object ("DEF_Em_GENEtaB_ptRel") at

TypeError: none of the 2 overloaded methods succeeded. Full details:
  attempt to access a null-pointer
  int TH1::GetMinimumBin(int& locmix, int& locmiy, int& locmiz) =>
    takes at least 3 arguments (0 given)

Error in <TH1F::DrawNormalized>: Sum of weights is null. Cannot normalize histogram: DEFMerged_all_GENEtaC_ptRel


In [45]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaEta for his in hsets], labels, norm=True, text=text)


[<ROOT.TH1F object ("DEF_all_GEN_deltaEta") at 0x7ff44e76ee60>, <ROOT.TH1F object ("DEF_all_GEN_deltaEta") at 0x7ff44e76dae0>]
[<ROOT.TH1F object ("DEF_all_GENEtaB_deltaEta") at 0x7ff44e5d0480>, <ROOT.TH1F object ("DEF_all_GENEtaB_deltaEta") at 0x7ff44e5d1cb0>]
[<ROOT.TH1F object ("DEF_all_GENEtaBC_deltaEta") at 0x7ff44f0563a0>, <ROOT.TH1F object ("DEF_all_GENEtaBC_deltaEta") at 0x7ff44e5d34b0>]
[<ROOT.TH1F object ("DEF_all_GENEtaBCD_deltaEta") at 0x7ff44e679b90>, <ROOT.TH1F object ("DEF_all_GENEtaBCD_deltaEta") at 0x7ff44e670630>]
[<ROOT.TH1F object ("DEF_all_GENEtaC_deltaEta") at 0x7ff44e686970>, <ROOT.TH1F object ("DEF_all_GENEtaC_deltaEta") at 0x7ff44e6873f0>]
[<ROOT.TH1F object ("DEF_all_GENEtaD_deltaEta") at 0x7ff44e5d23d0>, <ROOT.TH1F object ("DEF_all_GENEtaD_deltaEta") at 0x7ff44e5d4470>]
[<ROOT.TH1F object ("DEF_Em_GEN_deltaEta") at 0x7ff44e772bb0>, <ROOT.TH1F object ("DEF_Em_GEN_deltaEta") at 0x7ff44e772fa0>]
[<ROOT.TH1F object ("DEF_Em_GENEtaB_deltaEta") at 0x7ff44f053230>, 

In [21]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaPhi for his in hsets], labels, norm=True, text=text)


[<ROOT.TH1F object ("DEF_all_GEN_deltaPhi") at 0x7ff44e530670>, <ROOT.TH1F object ("DEF_all_GEN_deltaPhi") at 0x7ff44e530a60>]
[<ROOT.TH1F object ("DEF_all_GENEtaB_deltaPhi") at 0x7ff44e279600>, <ROOT.TH1F object ("DEF_all_GENEtaB_deltaPhi") at 0x7ff44e276560>]
[<ROOT.TH1F object ("DEF_all_GENEtaBC_deltaPhi") at 0x7ff44e3c9b20>, <ROOT.TH1F object ("DEF_all_GENEtaBC_deltaPhi") at 0x7ff44e3ca390>]
[<ROOT.TH1F object ("DEF_all_GENEtaBCD_deltaPhi") at 0x7ff44e532160>, <ROOT.TH1F object ("DEF_all_GENEtaBCD_deltaPhi") at 0x7ff44e532ab0>]
[<ROOT.TH1F object ("DEF_all_GENEtaC_deltaPhi") at 0x7ff44e533a60>, <ROOT.TH1F object ("DEF_all_GENEtaC_deltaPhi") at 0x7ff44e5348e0>]
[<ROOT.TH1F object ("DEF_all_GENEtaD_deltaPhi") at 0x7ff44dadb1a0>, <ROOT.TH1F object ("DEF_all_GENEtaD_deltaPhi") at 0x7ff44dadbbe0>]
[<ROOT.TH1F object ("DEF_Em_GEN_deltaPhi") at 0x7ff44e5359c0>, <ROOT.TH1F object ("DEF_Em_GEN_deltaPhi") at 0x7ff44e536ad0>]
[<ROOT.TH1F object ("DEF_Em_GENEtaB_deltaPhi") at 0x7ff44e3cbe60>, 

TypeError: none of the 2 overloaded methods succeeded. Full details:
  attempt to access a null-pointer
  int TH1::GetMinimumBin(int& locmix, int& locmiy, int& locmiz) =>
    takes at least 3 arguments (0 given)

Error in <TH1F::DrawNormalized>: Sum of weights is null. Cannot normalize histogram: DEFMerged_all_GENEtaC_deltaPhi


In [22]:
for tp in tps:
    for tp_sel in tp_select:
        for gen_sel in gen_select:
            hsets, labels, text = hplot.get_histo(histos.ClusterConeHistos, sample, ['PU0', 'PU200'], tp, tp_sel, gen_sel)            
            draw([his.h_deltaPhiVq for his in hsets], labels, text=text, options='COLZ', min_y=-1, max_y=2)


[<ROOT.TH2F object ("DEF_all_GEN_deltaPhiVq") at 0x7ff44e968400>, <ROOT.TH2F object ("DEF_all_GEN_deltaPhiVq") at 0x7ff44ccb2200>]
[<ROOT.TH2F object ("DEF_all_GENEtaB_deltaPhiVq") at 0x7ff449a7ee00>, <ROOT.TH2F object ("DEF_all_GENEtaB_deltaPhiVq") at 0x7ff44cb4c800>]
[<ROOT.TH2F object ("DEF_all_GENEtaBC_deltaPhiVq") at 0x7ff44cb4ac00>, <ROOT.TH2F object ("DEF_all_GENEtaBC_deltaPhiVq") at 0x7ff4494a4e00>]
[<ROOT.TH2F object ("DEF_all_GENEtaBCD_deltaPhiVq") at 0x7ff44e9e2200>, <ROOT.TH2F object ("DEF_all_GENEtaBCD_deltaPhiVq") at 0x7ff44ea5dc00>]
[<ROOT.TH2F object ("DEF_all_GENEtaC_deltaPhiVq") at 0x7ff44e9e3000>, <ROOT.TH2F object ("DEF_all_GENEtaC_deltaPhiVq") at 0x7ff44ea0d000>]
[<ROOT.TH2F object ("DEF_all_GENEtaD_deltaPhiVq") at 0x7ff44aaea000>, <ROOT.TH2F object ("DEF_all_GENEtaD_deltaPhiVq") at 0x7ff449533400>]
[<ROOT.TH2F object ("DEF_Em_GEN_deltaPhiVq") at 0x7ff44cb4a600>, <ROOT.TH2F object ("DEF_Em_GEN_deltaPhiVq") at 0x7ff44ccc5600>]
[<ROOT.TH2F object ("DEF_Em_GENEtaB_del